In [9]:
import numpy
from nltk.tokenize import word_tokenize
from PyLyrics import *
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import h5py as h5py

/Users/samirabulle/Documents/projects/rap_generator/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
all_lyrics = open("lyrics.txt","w+")

In [8]:
drake = """Tuscan Leather
Furthest Thing
Started from the Bottom
Wu-Tang Forever
Own It
Worst Behavior
From Time (featuring Jhené Aiko)
Hold On, We're Going Home (featuring Majid Jordan)
Connect
The Language
305 to My City (featuring Detail)
Too Much (featuring Sampha)
Pound Cake / Paris Morton Music 2 (featuring Jay-Z)
Come Thru (Deluxe edition bonus track)
All Me (featuring 2 Chainz and Big Sean) (Deluxe edition bonus track)
The Motion (featuring Sampha) (Best Buy bonus track)"""

j_cole = """Intro
January 28th
Wet Dreamz
03' Adolescence
A Tale of 2 Citiez
Fire Squad
St. Tropez
G.O.M.D.
No Role Modelz
Hello
Apparently
Love Yourz
Note to Self"""

kendrick = """Sherane a.k.a. Master Splinter's Daughter
Bitch, Don't Kill My Vibe
Backseat Freestyle
The Art of Peer Pressure
Money Trees (featuring Jay Rock)
Poetic Justice (featuring Drake)
good kid
m.A.A.d city (featuring MC Eiht)
Swimming Pools (Drank) (Extended Version)
Sing About Me, I'm Dying of Thirst
Real (featuring Anna Wise)
Compton (featuring Dr. Dre)
The Recipe (featuring Dr. Dre) (Deluxe edition track)
Black Boy Fly (Deluxe edition track)
Now or Never (featuring Mary J. Blige) (Deluxe edition track)
Collect Calls (featuring Kent Jamz) (iTunes deluxe edition track)
Swimming Pools (Drank) (iTunes deluxe edition track)
County Building Blues (Target bonus track)"""

drake_tracks = drake.split("\n")
j_cole_tracks = j_cole.split("\n")
kendrick_tracks = kendrick.split("\n")

for track in drake_tracks:
    all_lyrics.write(PyLyrics.getLyrics('Drake',track) + '\n')
for track in j_cole_tracks:
    all_lyrics.write(PyLyrics.getLyrics('J. Cole',track) + '\n')
for track in kendrick_tracks:
    all_lyrics.write(PyLyrics.getLyrics('Kendrick Lamar',track) + '\n')

/Users/samirabulle/Documents/projects/rap_generator/env/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/samirabulle/.pyenv/versions/3.6.4/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


ValueError: Song or Singer does not exist or the API does not have Lyrics

In [2]:
#all_lyrics.close()
raw_lyrics = open("lyrics.txt","r")
raw_lyrics = raw_lyrics.read()
raw_lyrics = raw_lyrics.lower()
#lyric_tokens = word_tokenize(raw_lyrics)

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/samirabulle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
chars = sorted(list(set(raw_lyrics)))
char_to_int = dict((c,i) for i, c in enumerate(chars))

In [5]:
n_chars = len(raw_lyrics)
n_vocab = len(chars)

In [6]:
seq_length = 100
data_input = []
data_output = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_lyrics[i:i + seq_length]
    seq_out = raw_lyrics[i + seq_length]
    data_input.append([char_to_int[word] for word in seq_in])
    data_output.append(char_to_int[seq_out])
n_patterns = len(data_input)

In [10]:
data_in = numpy.reshape(data_input, (n_patterns, seq_length, 1))

In [11]:
data_in = data_in/float(n_vocab)

In [12]:
data_out = np_utils.to_categorical(data_output)

In [13]:
model = Sequential()
model.add(LSTM(256, input_shape=(data_in.shape[1], data_in.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(data_out.shape[1], activation='softmax'))

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [16]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [17]:
model.fit(data_in, data_out, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
165120/165192 [============================>.] - ETA: 0s - loss: 2.9355
Epoch 00001: loss improved from inf to 2.93546, saving model to weights-improvement-01-2.9355.hdf5
165192/165192 [==============================] - 1036s 6ms/step - loss: 2.9355
Epoch 2/20
165120/165192 [============================>.] - ETA: 1s - loss: 2.7799
Epoch 00002: loss improved from 2.93546 to 2.77989, saving model to weights-improvement-02-2.7799.hdf5
165192/165192 [==============================] - 3563s 22ms/step - loss: 2.7799
Epoch 3/20
165120/165192 [============================>.] - ETA: 0s - loss: 2.7197
Epoch 00003: loss improved from 2.77989 to 2.71969, saving model to weights-improvement-03-2.7197.hdf5
165192/165192 [==============================] - 960s 6ms/step - loss: 2.7197
Epoch 4/20
165120/165192 [============================>.] - ETA: 0s - loss: 2.6684
Epoch 00004: loss improved from 2.71969 to 2.66828, saving model to weights-improvement-04-2.6683.hdf5
165192/165192 [========